# 知識點
## ResNet
- ResNet 為 2015年 ImageNet classificatoin的冠軍，其重點在於導入了殘差結構，降低梯度消失發生的可能性。
- ResNetV2 同樣由 Kaiming He 團隊提出，承襲 ResnetV1 的殘差概念，但在 Identity branch 與 Residual branch 上做了些許更改。<br>
最終結構如下圖最右：
<img src="https://i2.wp.com/img-blog.csdnimg.cn/20200523181526704.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3NpbmF0XzM0Njg2MTU4,size_16,color_FFFFFF,t_70" width=700>

可以注意到殘差區塊輸出時並沒有 ReLU，這是因為不希望限制它的輸出 (加 ReLU 就變成 $[0, \infty)$)。


另外 Identity branch 在 addition 之後也沒有接 BatchNormalization，這是因為 BN 層會改變 Identity branch 的訊息分佈，造成收斂速度下降。

論文中也使用了 Inception Block 中 1 x 1 kernel 壓縮深度的技巧，最後再用 1 x 1 kernel回放深度，藉此降低運算。

詳細請參考 [ResNetV2：ResNet深度解析](https://blog.csdn.net/lanran2/article/details/80247515)，裡面會解釋上圖的意義。

原論文為 [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) 與 [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)
      
##  InceptionV4  ＆ Inception-Resnet
- InceptionV4 的重點在於設計了多種不同的 Inception Block
- Inception-Resnet 的重點在於結合殘差結構，顯著加速 Inception 的訓練。
- Inception-Resnet 像是加入殘差結構的V3、V4版本：
    - Inception-ResNet v1 的運算量和 Inception v3 的接近。
    - Inception-ResNet v2 的運算量和 Inception v4 的接近。
- 其中加入殘差結構的版本收斂速度都大幅提升了許多。<br>
  論文中也有提到，相同的參數量下加入殘差結構並不一定能提升準度，但確實能提升模型收斂速度，並且能搭建更深的網路。
    
架構圖可參考原論文 [Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning](https://arxiv.org/abs/1602.07261)

# 延伸閱讀
[Residual blocks — Building blocks of ResNet](https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec)

## 『本次練習內容』
#### 學習如何搭建 Residual Block
####  學習如何搭建Inception-ResNet中的 Inception Block

## 『本次練習目的』
  #### 了解 Residual Block原理
  #### 了解如何結合Inception 與 Residual概念

---

## Part1

In [1]:
import numpy as np
from tensorflow.keras import models, layers
import tensorflow.keras.backend as K

![Incpeiton](img/ResNet_Structure.png)

## ResNetV1

In [2]:
def Residual_block(input_tensor, kernel_size, filters, stage, block):
    """Residual block v1"""
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = layers.Conv2D(filters1, kernel_size, name=conv_name_base+'2a')(input_tensor)
    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Conv2D(filters2, kernel_size, padding='same', name=conv_name_base+'2b')(x)
    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    
    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 參考ResNetV1 搭建 ResNetV2版本的Residual Block

In [3]:
def Residual_block_v2(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2a')(input_tensor)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters1, kernel_size, name=conv_name_base+'2a')(x)
    
    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters2, kernel_size, name=conv_name_base+'2b')(x)
    
    x = layers.add([x, input_tensor])
    return x

## 試試看自己設計一個先壓縮再回放的V2 Block

In [4]:
def Residual_block_v2(input_tensor, kernel_size, stage, block, reduce=96, ouput_size=128):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    x = layers.Conv2D(reduce, (1, 1), name=conv_name_base+'2a')(input_tensor)
    
    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2b')(input_tensor)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, kernel_size, name=conv_name_base+'2b')(x)
    
    x = layers.BatchNormalization(axis=3, name=bn_name_base+'2c')(input_tensor)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(128, kernel_size, name=conv_name_base+'2c')(x)    
    
    x = layers.Conv2D(output_size, (1, 1), name=conv_name_base+'2d')(x)
    
    x = layers.add([x, input_tensor])
    return x

---

## Part2

## Incpetion Block-A

![Incpeiton](img/Inception-ResNet-A.png)

## Incpetion Block-B

![Incpeiton](img/Inception-ResNet-B.png)

## Incpetion Block-C

![Incpeiton](img/Inception-ResNet-C.png)

In [5]:
def Conv2d_bn(x,filters,kernel_size,padding='same',strides=(1, 1),normalizer=True,activation='relu',name=None):
    if name is not None:
        conv_name = name + '_conv'
        bn_name = name + '_bn'
        act_name = name + '_act'
    else:
        conv_name = None
        bn_name = None
        act_name = None
    if K.image_data_format() == 'channels_first':
        bn_axis = 1
    else:
        bn_axis = 3
    x = layers.Conv2D(
        filters, kernel_size,
        strides=strides, padding=padding,
        use_bias=False, name=conv_name)(x)
    if normalizer:
        x = layers.BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation:
        x = layers.Activation(activation, name=act_name)(x)
    return x

In [6]:
def Residual_block(input_tensor, kernel_size, filters, stage, block):
    filters1, filters2, filters3 = filters
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=3, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=3, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x

## 參考上方Residual_block搭建 Inception-ResNet中的Inception Block

In [7]:
def inception_resnet_block(x, scale, block_type, activation='relu'):
    '''scale: scaling factor to scale the residuals (i.e., the output of
            passing `x` through an inception module) before adding them
            to the shortcut branch. Let `r` be the output from the residual branch,
            the output of this block will be `x + scale * r`.(簡單來說就是控制Residual branch的比例)'''
    if block_type == 'Incpetion_Block-A':
        branch_0 = layers.Conv2D(32, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(32, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(32, (3, 3), padding='same')(branch_1)
        branch_2 = layers.Conv2D(32, (1, 1), padding='same')(x)
        branch_2 = layers.Conv2D(48, (3, 3), padding='same')(branch_2)
        branch_2 = layers.Conv2D(64, (3, 3), padding='same')(branch_2)
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'Incpetion_Block-B':
        branch_0 = layers.Conv2D(192, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(128, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(160, (1, 7), padding='same')(branch_1)
        branch_1 = layers.Conv2D(192, (7, 1), padding='same')(branch_1)
        branches = [branch_0, branch_1]
    elif block_type == 'Incpetion_Block-C':
        branch_0 = layers.Conv2D(192, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(192, (1, 1), padding='same')(x)
        branch_1 = layers.Conv2D(224, (1, 3), padding='same')(branch_1) 
        branch_1 = layers.Conv2D(256, (3, 1), padding='same')(branch_1) 
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))
    mixed = layers.Concatenate(axis=3)(branches)
    
    '''確保輸入跟輸出深度相同'''
    up = Conv2d_bn(mixed, K.int_shape(x)[3], 1, activation=None)
    
    '''導入殘差結構，並給予權重'''
    x = layers.Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale, ##提示inputs[0]、inputs[1]
                      output_shape=K.int_shape(x)[1:],
                      arguments={'scale': scale},)([x,up])
    
    if activation is not None:
        x = layers.Activation(activation)(x)
    return x


In [8]:
img_input = layers.Input(shape=(224,224,32))
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
print(x)

Tensor("activation/Identity:0", shape=(None, 224, 224, 32), dtype=float32)


## 測試

In [9]:
img_input = layers.Input(shape=(224,224,32))
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-A', activation='relu')
print(x)

Tensor("activation_1/Identity:0", shape=(None, 224, 224, 32), dtype=float32)


In [10]:
img_input = layers.Input(shape=(224,224,32))
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-B', activation='relu')
print(x)

Tensor("activation_2/Identity:0", shape=(None, 224, 224, 32), dtype=float32)


In [11]:
img_input = layers.Input(shape=(224,224,32))
x = inception_resnet_block(img_input, 0.1, 'Incpetion_Block-C', activation='relu')
print(x)

Tensor("activation_3/Identity:0", shape=(None, 224, 224, 32), dtype=float32)
